<a href="https://colab.research.google.com/github/steffenschneider/MRI-tumor-detection/blob/main/mri_tumor_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries


In [1]:
# !pip install tensorflow

import os
import tensorflow as tf
import numpy as np
from tensorflow import keras
import tensorflow
from tensorflow import keras
from keras import layers, metrics, callbacks, applications, regularizers
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
print(os.getcwd())

/content


# Get data

In [12]:
if not os.path.exists('/content'):
    os.makedirs('/content')

import shutil
#shutil.rmtree('')

from google.colab import files
uploaded = files.upload()       # choose zip folder manually

data_dir = "./mri-tumor"  # contains "yes" and "no" folders

import zipfile
with zipfile.ZipFile('mri-data.zip', 'r') as zip_ref:
    zip_ref.extractall(data_dir)

shutil.rmtree(data_dir + '/brain_tumor_dataset')    # delete additional redundant files



Saving mri-data.zip to mri-data.zip


# split data into training and test

In [3]:
img_size = (224, 224)  # ResNet50 requires 224x224
batch_size = 25

# First split: 80% train, 20% validation and test
# color_mode="rgb" is needed for ResNet50, which expects 3 channels
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=img_size,        # image is resized here
    batch_size=batch_size,
    color_mode="rgb"
)

# color_mode="rgb" is needed for ResNet50, which expects 3 channels
temp_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=img_size,        # image is resized here
    batch_size=batch_size,
    color_mode="rgb"
)

# Second split: Split temp into 50% validation, 50% test (10% each of original)
n_batches = 1
val_ds = temp_ds.take(n_batches)     # 25 pictures for validation
test_ds = temp_ds.skip(n_batches)    # 25 pictures for testing

'''''
data_count = 0
for batch in test_ds:
    images, labels = batch
    data_count += len(images)
print(f"Sum of data / pictures: {data_count}")
'''


Saving mri-data.zip to mri-data.zip
